In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from pyspark.sql import SparkSession
from tqdm import tqdm
import time  # to simulate loading for tqdm

import os
import glob
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col, to_date, count, min, max, lit
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType
from pyspark.sql.functions import col, sum as spark_sum, when

import utils.bronze
import utils.silver
import utils.gold

print('Done Importing!')

Done Importing!


In [2]:
# # Create gold table and store to Google Cloud Storage
# gold_label_file_path = "Gold Layer/labels.csv"
# gold_label_gcs_path = f"gs://{bucket_name}/{gold_label_file_path}"

# try:
#     df_labels.to_csv(gold_label_gcs_path, index=False)
#     print("labels.csv Stored to Gold Layer Successfully! ✅")
# except Exception as e:
#     print(f"labels.csv Store Failed: {e}")

# Setup Hadoop
Go to your working directory: CS611_MLE_GROUP_PROJECT <br/>
Then, run this command in your VM terminal:
> wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.20/gcs-connector-hadoop3-2.2.20-shaded.jar -P jars/

<br/><br/>
Copy your google cloud application_default_credentials.json to this working directory, then can run the following commands:

In [2]:

spark = SparkSession \
    .builder \
    .config("spark.jars", "jars/gcs-connector-hadoop3-2.2.20-shaded.jar") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


25/06/15 15:30:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/15 15:31:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
# This is required if you are using service account and set true, 
spark._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.enable', 'true')
spark._jsc.hadoopConfiguration().set('google.cloud.auth.service.account.json.keyfile', "application_default_credentials.json")
# Following are required if you are using oAuth
# spark._jsc.hadoopConfiguration().set('fs.gs.auth.client.id', 'YOUR_OAUTH_CLIENT_ID')
# spark._jsc.hadoopConfiguration().set('fs.gs.auth.client.secret', 'OAUTH_SECRET')

# Partition Existing Bronze

In [4]:
# Setup Config
snapshot_date_str = "2015-01-01"

start_date_str = "2015-01-01"
end_date_str = "2017-03-31"

In [5]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
print("Done generate list of dates!")

Done generate list of dates!


In [6]:
dates_str_lst

['2015-01-01',
 '2015-02-01',
 '2015-03-01',
 '2015-04-01',
 '2015-05-01',
 '2015-06-01',
 '2015-07-01',
 '2015-08-01',
 '2015-09-01',
 '2015-10-01',
 '2015-11-01',
 '2015-12-01',
 '2016-01-01',
 '2016-02-01',
 '2016-03-01',
 '2016-04-01',
 '2016-05-01',
 '2016-06-01',
 '2016-07-01',
 '2016-08-01',
 '2016-09-01',
 '2016-10-01',
 '2016-11-01',
 '2016-12-01',
 '2017-01-01',
 '2017-02-01',
 '2017-03-01']

In [7]:
bucket_name = "cs611_mle"

### Bronze Member

In [9]:
### ~~~ Bronze Member ~~~
# create bronze datalake
bronze_member_directory = "datamart/bronze/member"

In [ ]:
# No need to be partitioned
utils.bronze.process_bronze_member(bucket_name, bronze_member_directory, spark)
print('Done Build Bronze Table - Member!')

### Bronze User Log

In [11]:
### ~~~ User Log ~~~
# create bronze datalake
# create multiple csv files 
bronze_userlog_directory = "datamart/bronze/userlog"

In [11]:

# run bronze backfill
for date_str in dates_str_lst:
    utils.bronze.process_bronze_userlog(date_str, bucket_name, bronze_userlog_directory, spark)
    
print('Done Build Bronze Table - userlog!')

2015-01-01row count: 2032255


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_01_01.csv


2015-02-01row count: 1812923


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_02_01.csv


2015-03-01row count: 2093862


...

saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_01_01.csv


2017-02-01row count: 2270306


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_02_01.csv


2017-03-01row count: 2587759


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv
Done Build Bronze Table - userlog!


# Partition Existing Silver

This should have been done in Bronze

### Silver Transaction

In [14]:
### ~~~ Transaction ~~~
# create silver datalake
# create multiple csv files 
bronze_transaction_directory = "datamart/bronze/transaction"

In [15]:

# run bronze backfill
for date_str in dates_str_lst:
    utils.bronze.process_bronze_transaction_partition(date_str, bucket_name, bronze_transaction_directory, spark)
print('Done Partition Silver (Actually Bronze) Table - transaction!')

2015-01-01row count: 97582


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_01_01.csv


2015-02-01row count: 88599


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_02_01.csv


2015-03-01row count: 110650


...

saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_01_01.csv


2017-02-01row count: 116238


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_02_01.csv


2017-03-01row count: 11247


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_03_01.csv
Done Partition Silver (Actually Bronze) Table - transaction!


# Process Feature Silver Table

### Member ~ Bronze to Silver Table

In [10]:
# create silver datalake
silver_member_directory = "datamart/silver/member/"

In [14]:
utils.silver.process_silver_table_member(bucket_name, bronze_member_directory, silver_member_directory, spark).toPandas().head()

loaded from: gs://cs611_mle/datamart/bronze/member/bronze_members.csv row count: 300000
+------+------+
|gender| count|
+------+------+
|  NULL|171532|
|  male| 65635|
|female| 62833|
+------+------+



saved to: gs://cs611_mle/datamart/silver/member//silver_member.parquet


,msno,city,gender,registered_via,registration_init_time
0,2dwuk0KQUnCnRvFnDGFyIslHLqG8lFHkyOvfRNwc5XI=,5,female,9,2012-03-09
1,Bx13MoR+nVJVVDsatg3fxInFupmO1jSzdkFKMA976t8=,13,female,7,2013-04-22
2,2ouWnOhbWxdFyjEy8qx/8lr15OzZ0PX9tr6xZP4Ihns=,1,na,7,2014-06-29
3,QvZtxnHz7yGh07QiFuq0RN1hL5WrzcbKlQTUiroEEyE=,13,na,9,2011-05-28
4,CUFR22oaKwIwzj4VMey3Mj76B5zawuo2PxsWheNfQ6Q=,5,male,9,2014-11-28


### Transaction ~ Partitioned_Silver to Actual Silver (processed)

In [11]:
actual_silver_transaction_directory = "datamart/silver/transaction/"

In [17]:
for date_str in dates_str_lst:
    utils.silver.process_silver_table_transaction(date_str, bucket_name, bronze_transaction_directory, actual_silver_transaction_directory, spark)

loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_01_01.csv row count: 97582


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_01_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_02_01.csv row count: 88599


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_02_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_03_01.csv row count: 110650


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_03_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_04_01.csv row count: 90444


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_04_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_05_01.csv row count: 74016


...

saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_11_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2016_12_01.csv row count: 138018


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_12_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_01_01.csv row count: 160929


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_01_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_02_01.csv row count: 116238


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_02_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_03_01.csv row count: 11247


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_03_01.parquet


### User Log ~ Bronze User Log to Silver User Log

In [12]:
silver_userlog_directory = "datamart/silver/userlog/"


# bronze_userlog_directory = "datamart/bronze/userlog"
# actual_silver_transaction_directory = "datamart/silver/transaction/"

In [10]:

for date_str in dates_str_lst:
    utils.silver.process_silver_table_userlog(date_str, bucket_name, bronze_userlog_directory, actual_silver_transaction_directory, silver_userlog_directory, spark)

loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_01_01.csv row count: 2032255


Also loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_02_01.csv row count: 1812923


25/06/15 00:58:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_01_01.parquet


loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_02_01.csv row count: 1812923


...

Also loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv row count: 2587759


25/06/15 01:05:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_02_01.parquet


loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv row count: 2587759
File gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_04_01.csv does not exist. Proceeding with single snapshot.


saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_03_01.parquet


# Gold ~ Feature Store

In [8]:
gold_feature_store_directory = "datamart/gold/feature_store/"

silver_userlog_directory = "datamart/silver/userlog/"
actual_silver_transaction_directory = "datamart/silver/transaction/"
silver_member_directory = "datamart/silver/member/"

In [ ]:
for date_str in dates_str_lst:
    utils.gold.process_gold_featurestore(date_str, bucket_name, actual_silver_transaction_directory, silver_userlog_directory, silver_member_directory, gold_feature_store_directory, spark)

loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_01_01.parquet row count: 97582
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_01_01.parquet row count: 90862
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


25/06/15 15:31:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2015_01_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_02_01.parquet row count: 88599
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_02_01.parquet row count: 83071
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2015_02_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_03_01.parquet row count: 110650
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_03_01.parquet row count: 103407
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2015_03_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_04_01.parquet row count: 90444
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_04_01.parquet row count: 82677
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


...

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2016_05_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_06_01.parquet row count: 113673
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2016_06_01.parquet row count: 105800
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2016_06_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_07_01.parquet row count: 123913
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2016_07_01.parquet row count: 115141
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2016_07_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_08_01.parquet row count: 150382
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2016_08_01.parquet row count: 139610
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2016_08_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_09_01.parquet row count: 128303
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2016_09_01.parquet row count: 119499
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2016_09_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_10_01.parquet row count: 152765
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2016_10_01.parquet row count: 140415
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 300000


In [14]:


# # =========== 修改成你的 GCS 路径 ============

# transaction_parquet_path = f"gs://{bucket_name}/{actual_silver_transaction_directory}/silver_transaction_2015_01_01.parquet"
# userlog_parquet_path = f"gs://{bucket_name}/{silver_userlog_directory}/silver_userlog_2015_01_01.parquet"
# member_parquet_path = f"gs://{bucket_name}/{silver_member_directory}/silver_member.parquet"


# # ======== 读取 Parquet 文件为 DataFrame ========
# df_transaction = spark.read.parquet(transaction_parquet_path)
# df_userlog = spark.read.parquet(userlog_parquet_path)
# df_member = spark.read.parquet(member_parquet_path)

# # ======== 展示前 20 行 ========
# print("=== Transaction Table Preview ===")
# # df_transaction.show(20, truncate=False)
# df_transaction.toPandas().head()


# print("=== Userlog Table Preview ===")
# # df_userlog.show(20, truncate=False)
# df_userlog.toPandas().head()

# print("=== Member Table Preview ===")
# # df_member.show(20, truncate=False)
# df_member.toPandas().head()

# # ======== 保存为 CSV 文件 ========
# # df_transaction.write.mode("overwrite").option("header", "true").csv(transaction_csv_path)
# # df_userlog.write.mode("overwrite").option("header", "true").csv(userlog_csv_path)
# # df_member.write.mode("overwrite").option("header", "true").csv(member_csv_path)

# # print("✅ Conversion complete. CSV files saved.")

=== Transaction Table Preview ===
=== Userlog Table Preview ===


=== Member Table Preview ===


,msno,city,gender,registered_via,registration_init_time
0,B5iaMV/EnzuN+YqGXvn0z8Vk3bj/VDGk6iJwjML3b28=,5,male,9,2004-07-24
1,46L2sK00j7owSpyqUExvgxiUs/cThzQS2f6VjUZN+Fs=,1,na,9,2015-07-28
2,JehndNgsKoc+qhGc6fJDKcHqvKQT4xM5jPzEaeUHE+4=,1,na,4,2016-04-09
3,RQ3eaqZn8iKvS9/qRQALXuo4LQabX5XxtrMgzMLXfK8=,1,na,9,2017-01-13
4,I2uQkLSFLjRL6ZPX5WBk/59Mpe0XSqfhBWS8sJ5oNyw=,1,na,7,2016-07-04


In [18]:
print("=== Userlog Table Preview ===")
# df_userlog.show(20, truncate=False)
df_userlog.toPandas().head(20)

=== Userlog Table Preview ===


,msno,membership_start_date,membership_expire_date,sum_completion_25,sum_completion_50,sum_completion_75,sum_completion_985,sum_completion_100,sum_songs,sum_total_secs,...,total_secs_first_7_days,total_secs_last_7_days,avg_seconds_per_songs,skip_ratio,songs_played_25_ratio,songs_played_50_ratio,songs_played_75_ratio,songs_played_985_ratio,songs_completion_ratio,last_first_7_days_total_secs_ratio
0,6tDJRqi0YnQSj5ZYBSVMSKnKJLnIxFUyU+weV0Rd5eY=,2015-01-18,2015-02-18,89,13,17,7,756,882,189953.270905,...,36205.222290,51777.932251,215.366520,0.115646,0.100907,0.014739,0.019274,0.007937,0.857143,1.430123
1,8UHoh579EZsyPDhyrHYiEEgdDuCUMG6BOwBXylccf9g=,2015-01-10,2015-02-10,48,59,26,7,225,365,61201.093079,...,15777.631958,2942.003052,167.674228,0.293151,0.131507,0.161644,0.071233,0.019178,0.616438,0.186467
2,daMHKj2zH4B9kxuxlj8xFAaHd7sKug1TAF2DAVBC+E4=,2015-01-12,2015-02-12,7,3,3,4,326,343,85284.844574,...,39731.336914,1469.901031,248.643862,0.029155,0.020408,0.008746,0.008746,0.011662,0.950437,0.036996
3,yCrji8OTMzuu0Dx9sOqx2cmG1TXipChAMGTG/5+EU7s=,2015-01-16,2015-02-16,64,36,41,26,62,229,31074.318115,...,6582.186096,5040.774094,135.695712,0.436681,0.279476,0.157205,0.179039,0.113537,0.270742,0.765821
4,FoTdaG+giNMhn7icv4qUiYiuK7+ysxcZXSRfYsyg9Dc=,2015-01-04,2015-02-03,64,20,19,13,716,832,194388.747314,...,36482.972717,73684.392273,233.640321,0.100962,0.076923,0.024038,0.022837,0.015625,0.860577,2.019692
5,Rz6VfCpaEiy/0BO3YosgzeuPHorGqJgv6ql8P6oE57o=,2015-01-05,2015-02-04,534,224,174,185,1164,2281,374239.449707,...,79158.923340,87792.549072,164.068150,0.332310,0.234108,0.098203,0.076282,0.081105,0.510302,1.109067
6,HT+2ile+EngqX3IO8TPL2Jzf/bcswhSFsQfttMlm/J4=,2015-01-09,2015-02-08,178,15,4,5,475,677,118273.041901,...,30222.870117,23116.510895,174.701687,0.285081,0.262925,0.022157,0.005908,0.007386,0.701625,0.764868
7,R94Rn++ZyYTWpKulrcE/C/lCUlN80qVR3fWrkkdM/Uk=,2015-01-03,2015-02-02,18,5,1,4,720,748,184328.976776,...,52496.966644,14459.742676,246.429113,0.030749,0.024064,0.006684,0.001337,0.005348,0.962567,0.275440
8,0hMhhZmpHYlk18IQj8Nx+QA9Zwe1/bGC3RvSML3qv8M=,2015-01-04,2015-02-04,28,2,2,5,333,370,80468.070480,...,0.000000,203.759995,217.481272,0.081081,0.075676,0.005405,0.005405,0.013514,0.900000,20376.999451
9,jl1a1KTvoQLfjHBX9vVNCpdNVT4ba0zkKwLv149zWgc=,2015-01-13,2015-02-12,128,41,36,37,365,607,100571.548218,...,34729.021484,32512.581909,165.686241,0.278418,0.210873,0.067545,0.059308,0.060956,0.601318,0.936179


In [17]:
print("=== Transaction Table Preview ===")
# df_transaction.show(20, truncate=False)
df_transaction.toPandas().head(20)

=== Transaction Table Preview ===


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,membership_start_date,discount_ratio
0,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,41,30,149.0,149.0,1,2015-01-08,2015-02-09,0,2015-01-10,0.000000
1,++E1waHrjSGFrLK4FgYHlaQgZ22Fn2fuoc929CTPtlA=,40,31,149.0,149.0,1,2015-01-08,2015-02-07,0,2015-01-07,0.000000
2,++E1waHrjSGFrLK4FgYHlaQgZ22Fn2fuoc929CTPtlA=,40,31,149.0,149.0,1,2015-01-11,2015-02-07,1,2015-01-07,0.000000
3,++FojTWzKjhz7uxsZXEN3KBpxSmSvPr2j9KEuLzVAm4=,39,31,149.0,149.0,1,2015-01-31,2015-03-02,0,2015-01-30,0.000000
4,++GVWCJ/8c4RVSUmn5mrt3VFGLUfmYIGVJ/SHLWPACw=,41,30,149.0,119.0,1,2015-01-25,2015-02-26,0,2015-01-27,0.201342
5,++IU7zZpfAGO1qdHBDkHlI+6UzjMMpd/JiaY87ykRyo=,41,30,149.0,119.0,1,2015-01-20,2015-02-21,0,2015-01-22,0.201342
6,++KbErD/TJoTzWzoMQzvaHHnRPE5GZLjXR2YbfbJ+ow=,41,30,149.0,119.0,1,2015-01-01,2015-02-01,0,2015-01-02,0.201342
7,++KbErD/TJoTzWzoMQzvaHHnRPE5GZLjXR2YbfbJ+ow=,41,30,149.0,119.0,1,2015-02-01,2015-03-01,0,2015-01-30,0.201342
8,++VNRMQoGY6E5ontEaWuAPUqVk/W1vbSDMzRfc3TQbM=,41,30,149.0,119.0,1,2015-01-17,2015-02-17,0,2015-01-18,0.201342
9,++ctvOaat5GA9vNamCSPGS2NtyaKjQWF1qjpBBq/nI8=,37,31,149.0,149.0,1,2015-01-13,2015-02-13,0,2015-01-13,0.000000
